In [1]:
import numpy as np 
import pandas as pd
# import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/My Drive/Amazon Fine Food Review/Final_Vivek.csv')
df.head()

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment,CleanedText
0,150523,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,5,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,1,b'witti littl book make son laugh loud recit c...
1,150500,150501,0006641040,AJ46FKXOVC7NR,Nicholas A Mesiano,2,2,5,940809600,This whole series is great way to spend time w...,I can remember seeing the show when it aired o...,1,b'rememb see show air televis year ago child s...
2,451855,451856,B00004CXX9,AIUWLEQ1ADEG5,Elizabeth Medina,0,0,5,944092800,Entertainingl Funny!,Beetlejuice is a well written movie ..... ever...,1,b'beetlejuic well written movi everyth excel a...
3,230284,230285,B00004RYGX,A344SMIA5JECGM,Vincent P. Ross,1,2,5,944438400,A modern day fairy tale,"A twist of rumplestiskin captured on film, sta...",1,b'twist rumplestiskin captur film star michael...
4,451854,451855,B00004CXX9,AJH6LUC1UT1ON,The Phantom of the Opera,0,0,5,946857600,FANTASTIC!,Beetlejuice is an excellent and funny movie. K...,1,b'beetlejuic excel funni movi keaton hilari wa...


In [4]:
df = df[df['Sentiment'] != 0]

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
final = df.sort_values('Time',axis= 0,inplace = False , na_position = 'last',ascending = True)
X = final['CleanedText'].values
y = final['Sentiment'].values

In [7]:
X_train ,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,stratify = y)

In [8]:
# Logistic regression on bow

In [9]:
count_vect = CountVectorizer() #in scikit-learn
bow_train = count_vect.fit_transform(X_train)
bow_test = count_vect.transform(X_test)
#count_vect.get_feature_names()
bow_train.shape

(228617, 57871)

In [10]:
tunned_param = [{'C' : [10** -4, 10**-3,10**-2,10**-1,1,10,100,1000,10000]}]
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
model  = GridSearchCV(LogisticRegression(class_weight = 'balanced'),tunned_param , scoring = 'accuracy',cv = 10,n_jobs = -1)
model.fit(bow_train,y_train)
print(model.best_estimator_)
pred = model.predict(bow_test)
print('Accuracy ',accuracy_score(y_test,pred)*100)

LogisticRegression(C=0.1, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Accuracy  89.17829330774963


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [11]:
# Logistic regression on tfidf

In [12]:
tfidf_vect = TfidfVectorizer()
tfidf_train = tfidf_vect.fit_transform (X_train)
tfidf_test = tfidf_vect.transform(X_test)

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean = False)
X_tr = sc.fit_transform(tfidf_train)
X_te = sc.transform(tfidf_test)
param = [{'C':[10**i for i in range(-3,4)]}]
gd = GridSearchCV(LogisticRegression(class_weight = 'balanced',penalty = 'l2'),param,cv = 10, scoring = 'accuracy',n_jobs = -1)
gd.fit(X_tr,y_train)
print(gd.best_estimator_)
pred = gd.predict(X_te)
print('Accuracy is ',accuracy_score(y_test,pred)*100)

LogisticRegression(C=0.001, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Accuracy is  88.21992467773707


In [14]:
# Logistic Regression on Avg W2Vec

In [34]:
list_of_sent_train = []
for i in X_train:
    sent = []
    i = i.decode('utf-8')
    for word in i.split():
        
        sent.append(word)
        
    list_of_sent_train.append(sent)

AttributeError: ignored

In [33]:
#Name = b'My name is Vivek'
#Name = Name.decode('utf-8')
#sample = []
#for word in Name.split():
#    sample.append(word)
#print(sample)

['My', 'name', 'is', 'Vivek']


In [41]:
for word in X_train:
    print(word.decode('utf-8'))

AttributeError: ignored

In [ ]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(list_of_sent_train,min_count = 5,size = 50,workers = 4)
sent_vectors_train = []
for sent in list_of_sent_train:
    sent_vec = np.zeros(50)
    cnt_word = 0
    for word in sent:
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_word += 1
        except:
            pass
    sent_vec /= cnt_word
    sent_vectors_train.append(sent_vec)
print(len(sent_vectors_train))

In [ ]:
list_of_sent_test = []
for i in X_test:
    sent = []
    for word in i.split():
        sent.append(word.decode('utf-8'))
    list_of_sent_test.append(sent)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from gensim.models import Word2Vec
w2v_model = Word2Vec(list_of_sent_test,min_count = 5,size = 50,workers = 4)
sent_vectors_test = []
for sent in list_of_sent_test:
    sent_vec = np.zeros(50)
    cnt_word = 0
    for word in sent:
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_word += 1
        except:
            pass
    sent_vec /= cnt_word
    sent_vectors_test.append(sent_vec)
print(len(sent_vectors_test))

In [ ]:
np.where(np.isnan(sent_vectors_test))